In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install timm segmentation_models_pytorch rasterio torchmetrics
!apt install -y zip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zip is already the newest version (3.0-12build2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
import torch.nn.functional as F

In [4]:
params = {
    'backbone': 'tf_efficientnetv2_b2',
    'weights': 'imagenet', # 'noisy-student',
    'learning_rate': 0.001,
    'epochs': 50,
    'batch_size': 30,
    'num_workers': 12,
    'device': torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'),
    'div_factor':10,
    'final_div_factor':100,
    'threshhold':0.5
}

## Functions

We define a function on which we can measure the accuracy of our model.

In [5]:
def sort_paths_by_number(paths):
    return sorted(paths, key=lambda x: int(x.split('_')[-1].split('.tif')[0]))

In [6]:
def calculate_f1_score(pred, true, threshold=0.5):
    """
    Compute the F1 score, also known as balanced F-score or F-measure.

    Parameters:
    - true: ground truth (correct) labels, a tensor of shape [N, H, W].
    - pred: predicted labels, a tensor of shape [N, 1, H, W].
    - threshold: the prediction threshold, defaults to 0.5.

    Returns:
    - F1 score
    """

    # Flatten the tensors
    true = true.numpy().flatten()
    pred = pred.numpy().flatten()

    # Calculate F1 score
    f1 = f1_score(true, pred)

    return f1

In [7]:
def intersection_and_union(pred, true):
    """
    Calculates intersection and union for a batch of images.

    Args:
        pred (torch.Tensor): a tensor of predictions
        true (torc.Tensor): a tensor of labels

    Returns:
        intersection (int): total intersection of pixels
        union (int): total union of pixels
    """
    pred = pred.squeeze(1)
    valid_pixel_mask = true.ne(255)  # valid pixel mask
    true = true.masked_select(valid_pixel_mask)
    pred = pred.masked_select(valid_pixel_mask)

    # Convert to CPU and NumPy for logical operations
    true = true.numpy()
    pred = pred.numpy()

    # Intersection and union totals
    intersection = np.logical_and(true, pred)
    union = np.logical_or(true, pred)
    return intersection.sum(), union.sum()

In [8]:
def save_model(epoch,model,ckpt_path='./',name='tf_efficientnetv2_b2_crop',val_iou=0):
    path = os.path.join(ckpt_path, '{}_wo_ca.pth'.format(name))
    torch.save(model.state_dict(), path, _use_new_zipfile_serialization=False)

def load_model(model,ckpt_path):
    state = torch.load(ckpt_path)
    model.load_state_dict(state)
    return model

## Loss Functions

In [9]:
class FocalLoss(torch.nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=2,reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights
        self.bce = torch.nn.BCEWithLogitsLoss(reduction="none")

    def forward(self, input, target):
        #ce_loss = F.binary_cross_entropy_with_logits(input.squeeze(1), target.long(),reduction=self.reduction,weight=self.weight)
        ce_loss = self.bce(input.squeeze(1), target.squeeze(1))
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [10]:
class XEDiceLoss(torch.nn.Module):
    """
    Computes (0.5 * CrossEntropyLoss) + (0.5 * DiceLoss).
    """

    def __init__(self):
        super().__init__()
        self.xe = torch.nn.BCEWithLogitsLoss(reduction="none")
        #self.xe = FocalLoss(reduction='none')

    def forward(self, pred, true):

        pred = pred.squeeze(1)
        valid_pixel_mask = true.ne(255)  # valid pixel mask

        # Cross-entropy loss
        temp_true = torch.where((true == 255), 0, true)  # cast 255 to 0 temporarily
        xe_loss = self.xe(pred, temp_true.float())
        xe_loss = xe_loss.masked_select(valid_pixel_mask).mean()

        # Dice loss

        pred = pred.sigmoid() #torch.softmax(pred, dim=1)[:, 1]

        pred = pred.masked_select(valid_pixel_mask)
        true = true.masked_select(valid_pixel_mask)

        dice_loss = 1 - (2.0 * torch.sum(pred * true) + 1e-7) / (torch.sum(pred + true) + 1e-7)

        #print(xe_loss,dice_loss,(2.0 * torch.sum(pred * true) + 1e-7))

        return (0.5 * xe_loss) + (0.5 * dice_loss)

## Data
We create the Dataset and Dataloader

In [11]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split

import glob
import pandas as pd
import tifffile
import os

In [12]:
PATH_SRC = f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/src'
PATH_OUTPUT = f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/submit'
PATHS_VAL = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/evaluation_true_color/evaluation_true_color_*.tif'))
PATHS_VAL_MASK = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/sample/evaluation_mask_*.tif'))
PATHS_TRAIN = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_*.tif'))
PATHS_TRAIN_MASK = sort_paths_by_number(glob.glob(f'/content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_*.tif'))

In [13]:
class DataTransform():
    def __init__(self):
        self.data_transform = {
            "train": A.Compose(
                [
                  A.ShiftScaleRotate(shift_limit=0.0625,rotate_limit=15,p=0.5),
                  A.GridDistortion(p=0.35),
                  A.RandomCrop(384,384),
                  A.HorizontalFlip(p=0.5),
                  A.VerticalFlip(p=0.5),
                  A.GaussianBlur(p=0.25),
                  # A.PadIfNeeded(min_height=1024, min_width=1024, border_mode=0),
                  # A.Resize(512, 512, interpolation=1, p=1),
                  ToTensorV2()
                ]
            ),
            "val": A.Compose(
                [
                  A.PadIfNeeded(min_height=1024, min_width=1024, border_mode=0),
                  # A.Resize(512, 512, interpolation=1, p=1),
                  ToTensorV2()
                ]
            )
        }

    def __call__(self, phase, img, mask):
      return self.data_transform[phase](image=img, mask=mask)

In [14]:
class CloudDataset(Dataset):
  def __init__(self, df, phase, transform):
    self.data = df
    self.phase = phase
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def normalize_data(self, image_tiff):
    # Normalize the image for better visualization
    for i in range(3):  # Assuming the image has 3 channels
        image_tiff[:,:,i] = (image_tiff[:,:,i] - image_tiff[:,:,i].min()) / (image_tiff[:,:,i].max() - image_tiff[:,:,i].min())

    return image_tiff

  def __getitem__(self, index):
    row = self.data.iloc[index]
    true_color_PATH = row['true_color']
    mask_PATH = row['mask']
    img = tifffile.imread(true_color_PATH)
    img = self.normalize_data(img.astype(np.float32))
    mask = tifffile.imread(mask_PATH)
    mask = mask.astype(np.float32)

    img_and_mask = self.transform(self.phase, img, mask)
    img = img_and_mask["image"]
    mask = img_and_mask["mask"]

    return img, mask

In [15]:
df = pd.DataFrame({
    'true_color': PATHS_TRAIN,
    'mask': PATHS_TRAIN_MASK
})

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
print(df.head())

df_train, df_val = train_test_split(df, test_size=0.2, random_state=2)

train_dataset = CloudDataset(df_train, phase="train", transform=DataTransform())
train_dataloader = DataLoader(
    train_dataset,
    batch_size=params['batch_size'],
    shuffle=True,
    num_workers=params['num_workers'],
    pin_memory=True,
    persistent_workers=True
)


val_dataset = CloudDataset(df_val, phase="val", transform=DataTransform())
val_dataloader = DataLoader(
    val_dataset,
    batch_size=4,#params['batch_size'],
    shuffle=True,
    num_workers=params['num_workers'],
    pin_memory=True,
    persistent_workers=True
)

dataloaders_dict = {
    'train': train_dataloader,
    'val': val_dataloader
}

                                                                                        true_color                                                                                 mask
0  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_0.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_0.tif
1  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_1.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_1.tif
2  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_2.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_2.tif
3  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/train_true_color_3.tif  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_mask/train_mask_3.tif
4  /content/drive/MyDrive/Projekte/Solafune/Cloud_satelite/train_true_color/trai

<ipython-input-15-0eca7a5c4c84>:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


## Model
We implement the cloud detection Model based on the Microsoft Competion Winner: https://github.com/drivendataorg/cloud-cover-winners/blob/main/1st%20place/src/unet_efficient_net_b1_train_fp16.py

In [16]:
from segmentation_models_pytorch.decoders.unet.model import UnetDecoder
from segmentation_models_pytorch.base import SegmentationHead
import torch
import torch.nn as nn
import timm

In [17]:
class Net(nn.Module):
    def __init__(self,params):
        super(Net, self).__init__()
        norm_cfg = dict(type='BN', requires_grad=True)
        self.backbone = timm.create_model('tf_efficientnetv2_b2', features_only=True,
                                          out_indices=[0,1,2,3],pretrained=True)
        self.decode_head = UnetDecoder(
                            encoder_channels=[16, 32, 56, 120],
                            decoder_channels=[16, 32, 56, 120],
                            n_blocks=4,
                            use_batchnorm=True,
                            center=False,
                            attention_type=None
                        )

        self.segment_classifier = SegmentationHead(56,1,upsampling=4)


    def forward(self,image):
        x = self.backbone(image)
        x=self.decode_head(*x)
        x=self.segment_classifier(x)
        x = F.interpolate(x, image.shape[-2:], mode="bilinear", align_corners=True)
        return x


In [18]:
model = Net(params)
model.to(params['device'])

Net(
  (backbone): EfficientNetFeatures(
    (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): ConvBnAct(
          (conv): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (drop_path): Identity()
        )
        (1): ConvBnAct(
          (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
       

## Training

In [19]:
from tqdm import tqdm
import matplotlib.pyplot as plt

In [20]:
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])

scheduler = torch.optim.lr_scheduler.OneCycleLR(
      optimizer,
      steps_per_epoch = 42,
      pct_start = 0.1,
      max_lr = params['learning_rate'],
      epochs = params['epochs'],
      div_factor = params['div_factor'],
      final_div_factor = params['final_div_factor'],
      verbose = True
    )

# criterion = FocalLoss()
criterion = XEDiceLoss()

Adjusting learning rate of group 0 to 1.0000e-04.


In [21]:
def train(model, img, mask):
  model.train()
  optimizer.zero_grad()

  preds = model(img)
  loss = criterion(preds, mask)
  # loss = criterion(preds.squeeze(dim=1), mask.squeeze(dim=1))
  loss.backward()
  optimizer.step()
  loss = loss.item()

  # Calculate validation IOU (global)
  preds = (preds.detach().sigmoid().cpu() > params['threshhold']) *1
  mask = mask.detach().cpu()

  # plt.imshow(preds[0][0], cmap='gray')
  # plt.colorbar()
  # plt.title("pred")
  # plt.show()

  # plt.imshow(mask[0], cmap='gray')
  # plt.colorbar()
  # plt.title("mask")
  # plt.show()


  intersection, union = intersection_and_union(preds, mask)
  f1_score = calculate_f1_score(preds, mask)

  return loss, intersection, union, f1_score

In [22]:
def val(model, img, mask):
  model.eval()
  with torch.no_grad():
    preds = model(img)
    loss = criterion(preds, mask)
    # loss = criterion(preds.squeeze(dim=1), mask.squeeze(dim=1))
    loss = loss.item()

    preds = (preds.detach().sigmoid().cpu() > params['threshhold']) *1 #torch.softmax(preds, dim=1)[:, 1]
    mask = mask.detach().cpu()
    intersection, union = intersection_and_union(preds, mask)
    f1_score = calculate_f1_score(preds, mask)

  return loss, intersection, union, f1_score

Delete GPU-RAM

In [23]:
import gc  # Garbage Collector
gc.collect()
torch.cuda.empty_cache()

In [24]:
best_iou = 0

for epoch in range(1, params['epochs']+1):
    print(f'\nEpoch: {epoch}')
    for phase in ['train', 'val']:
        total_intersection=0
        total_union=0
        total_loss=0
        total_f1=0

        pbar = tqdm(enumerate(dataloaders_dict[phase]),
                              total=len(dataloaders_dict[phase]),
                              leave=True,
                              dynamic_ncols=True)
        for i, (img, mask) in pbar:
            img, mask = img.to(params['device'], non_blocking=True), mask.to(params['device'], non_blocking=True)

            # training
            if phase == 'train':
                loss, intersection, union, f1 = train(model, img, mask)
            # validation
            else:
                loss, intersection, union, f1 = val(model, img, mask)

            # free GPU
            del img
            del mask

            total_intersection += intersection
            total_union += union
            total_loss += loss
            iou = total_intersection/total_union
            total_f1 += f1
            pbar.set_postfix({'iou': iou, 'loss': total_loss/(i+1), 'f1': total_f1/(i+1)})

        total_loss /= len(dataloaders_dict[phase])
        total_f1 /= len(dataloaders_dict[phase])

        if phase == 'train':
          scheduler.step()

        print(f'    Phase: {phase} -->  loss={total_loss} iou={iou} f1={total_f1}')

    # save best model
    if iou > best_iou:
      print(f'Saving model with iou {iou}')
      save_model(epoch, model, ckpt_path=PATH_SRC, val_iou=iou)
      best_iou = iou
    else:
      print(f'Not saving for iou {iou}')




Epoch: 1


100%|██████████| 27/27 [01:42<00:00,  3.79s/it, iou=0.362, loss=0.583, f1=0.533]

Adjusting learning rate of group 0 to 1.0005e-04.
    Phase: train -->  loss=0.5825601308434097 iou=0.36243740180659184 f1=0.532691695500523



100%|██████████| 50/50 [01:55<00:00,  2.31s/it, iou=0.424, loss=0.56, f1=0.527]


    Phase: val -->  loss=0.5600672638416291 iou=0.424030829043672 f1=0.5265316039053128
Saving model with iou 0.424030829043672

Epoch: 2


...

    Phase: val -->  loss=0.140995199624449 iou=0.8280996802455667 f1=0.8351346522104435
Not saving for iou 0.8280996802455667

Epoch: 50


100%|██████████| 27/27 [01:15<00:00,  2.81s/it, iou=0.849, loss=0.0992, f1=0.915]


Adjusting learning rate of group 0 to 2.2122e-04.
    Phase: train -->  loss=0.09918201963106792 iou=0.848582831675319 f1=0.9149674366914097


100%|██████████| 50/50 [01:55<00:00,  2.31s/it, iou=0.807, loss=0.139, f1=0.857]

    Phase: val -->  loss=0.13850296899676323 iou=0.8074505690253813 f1=0.8567044684912156
Not saving for iou 0.8074505690253813


### Results
loss=0.12502740886062383 iou=0.8384907222942675 f1=0.8704478890199857